In [1]:
import os
import gc
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import cuvis
import glob
from collections import defaultdict
import cv2
from torch.utils.data import Dataset
from pycocotools import mask as maskUtils
from enum import Enum
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report
from sklearnex import patch_sklearn
from skorch import NeuralNetClassifier, NeuralNetRegressor
from skorch.callbacks import LRScheduler, EarlyStopping, Checkpoint, ProgressBar
import torch
import skorch
import pickle
import numpy as np
from scipy.stats import chi2
from sklearn import preprocessing 
from torch import nn
import torch.nn.functional as F
from pretty_confusion_matrix import pp_matrix_from_data
import concurrent.futures
# Set us to use the GPU when appropriate
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} for pytorch models...')
%matplotlib inline

Using cuda for pytorch models...


### Transform .cu3s Dataset to .npy

This will save approximately 35 seconds per load operation (and only needs to be run once)

In [2]:
import concurrent.futures
import time
from tqdm.notebook import tqdm

def process_item(cube):
    # Load the cube to the numpy array
    cube_data = cuvis.SessionFile(cube).get_measurement(0).cube.to_numpy()
    # Save the measurement as numpy format
    np.save(cube.replace('.cu3s', '.npy'), cube_data)
    # Remove the data
    os.remove(cube)

def parallel_transform_with_progress(data, transform_fn, num_workers=4):
    """
    Parallelizes a transformation function with progress tracking using tqdm.

    Parameters:
        data (iterable): The input data to transform.
        transform_fn (callable): A function to apply to each element.
        num_workers (int): Number of parallel processes.

    Returns:
        list: Transformed data.
    """
    results = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Submit jobs and wrap with tqdm for progress tracking
        futures = {executor.submit(transform_fn, item): item for item in data}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(data), desc="Processing"):
            results.append(future.result())  # Collect result when done
    return results


data = list(glob.glob('/home/nathaniel/Downloads/yogurt_data_deploy/train_eval/wednesday/*.cu3s'))
transformed_data = parallel_transform_with_progress(data, process_item, num_workers=4)
print(transformed_data)

Processing: 0it [00:00, ?it/s]

[]


In [3]:
class CubeDataset(Dataset):
    """
    A PyTorch Dataset for a directory of hyperspectral cubes.
    
    Each hyperspectral image is stored as a .npy file with shape
        (height, width, bands).
    
    When __getitem__ is called, the corresponding hyperspectral cube is loaded 
    from disk (lazy loading) and returned (optionally with a transform).
    """
    def __init__(self, data_dir, subset: int =4, eval=False, RGB=False, SWIR=False, crop=1):
        """
        Parameters:
            data_dir (str): Path to the directory containing hyperspectral cube files (.npy).
            subset (int): Amount to downsample the shuffled hyperspectral data
        """
        self.data_dir = data_dir
        # Ablation - use RGB
        self.RGB = RGB
        # Ablation - use SWIR
        self.SWIR = SWIR
        # Crop - exclude edges
        self.crop = crop
        # List all .npy files in the directory, sorted for reproducibility.
        self.eval=eval
        if not eval:
            self.filenames = sorted([f for f in glob.glob(data_dir) if (f.endswith('.npy') and ('_ok_ok' in f) and ('pred' not in f))])
            print(f'Training from {len(self.filenames)} samples')
        else:
            self.filenames = sorted([f for f in glob.glob(data_dir) if ((f.endswith('.npy') and 'prediction' not in f and (('_ok_ok' in f) or ('_nok_ok' in f) or ('_ok_nok' in f) or ('_nok_nok' in f))))])
            print(f'Evaluating {len(self.filenames)} samples')
        if not self.filenames:
            raise ValueError("No .npy files found in the provided data directory.")
        # Apply the subset
        self.subset = subset
        # Calculate dark ref
        self.calculate_dark_ref(data_dir)
        # Calculate white ref
        self.calculate_white_ref(data_dir)

    def calculate_dark_ref(self, data_dir):
        filenames = sorted([f for f in glob.glob(data_dir) if (f.endswith('.npy') and '_ref_dark' in f)])
        print(f'Found {len(filenames)} dark reference files')
        cubes = np.array([np.load(cube)[self.crop:-self.crop:self.subset, self.crop:-self.crop:self.subset, :] for cube in filenames])
        self.dark_ref = np.mean(cubes, axis=0)
        print(f'Calculated dark reference with a size of {self.dark_ref.shape}')

    def calculate_white_ref(self, data_dir):
        filenames = sorted([f for f in glob.glob(data_dir) if (f.endswith('.npy') and '_ref_white' in f)])
        print(f'Found {len(filenames)} white reference files')
        cubes = np.array([np.load(cube)[self.crop:-self.crop:self.subset, self.crop:-self.crop:self.subset, :] for cube in filenames])
        self.white_ref = np.mean(cubes, axis=0)
        print(f'Calculated white reference with a size of {self.white_ref.shape}')

    def reflectance(self, cube):
        return np.clip(
            np.nan_to_num((cube - self.dark_ref) / (self.white_ref/0.40 - self.dark_ref)),
            0, 1.00).astype(np.float32)

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self):
            raise IndexError("Index out of range")
        
        fname = self.filenames[idx]
        file_path = os.path.join(self.data_dir, fname)
        
        # Load the entire hyperspectral cube lazily.
        cube = np.load(file_path)[self.crop:-self.crop:self.subset, self.crop:-self.crop:self.subset, :]
        
        # Reflectance calibrate_cube
        cube = self.reflectance(cube)
        height, width, n_bands = cube.shape
        
        # Flatten the cube: each row is a pixel's spectrum.
        # The resulting shape will be (height * width, bands)
        data = cube.reshape(-1, n_bands).astype(np.float32)
        if self.RGB:
            data = data[:,:3]
        if self.SWIR:
            data = data[:,3:]
        if not self.eval:
            # Subset the data
            np.random.shuffle(data)
            return data, data # We need both for x and y values
        else:
            return data, fname, (height, width, n_bands)

## Define Skorch Autoencoder Models

In [4]:
# Define the PyTorch model
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim: int, wave: int):
        super(Autoencoder, self).__init__()
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(wave, 150),
            nn.ReLU(),
            nn.Linear(150, 100),
            nn.ReLU(),
            nn.Linear(100, 75),
            nn.ReLU(),
            nn.Linear(75, 50),
            nn.ReLU(),
            nn.Linear(50, encoding_dim)
        )
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 75),
            nn.ReLU(),
            nn.Linear(75, 100),
            nn.ReLU(),
            nn.Linear(100, wave),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class AutoencoderSmall(nn.Module):
    def __init__(self, encoding_dim: int, wave: int):
        super(AutoencoderSmall, self).__init__()
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(wave, 5),
            nn.ReLU(),
            nn.Linear(5, 3),
            nn.ReLU(),
            nn.Linear(3, encoding_dim)
        )
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 3),
            nn.ReLU(),
            nn.Linear(3, 5),
            nn.ReLU(),
            nn.Linear(5, wave),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


# Create early stopping patience
early_stopping = EarlyStopping(monitor='valid_loss', patience = 10, threshold = 0.01, threshold_mode='rel', lower_is_better=True)


class CosSpectralAngleLoss(nn.Module):
    def __init__(self):
        super(CosSpectralAngleLoss, self).__init__()
    def forward(self, y, y_reconstructed):
        # Normalize y and y_reconstructed along the feature dimension
        # This should normalize along the length of the spectral vector
        epsilon = 1e-8
        normalize_r = torch.sqrt(torch.sum(y_reconstructed**2, dim=1)) + epsilon
        # This should also normalize along the length of the spectral vector
        normalize_t = torch.sqrt(torch.sum(y**2, dim=1))
        # Compute cosine similarity between the normalized vectors
        numerator = torch.sum((y * y_reconstructed), dim=1)
        denominator = normalize_r * normalize_t
        cosine_similarity = numerator / denominator
        # Compute the spectral angle in radians
        spectral_angle = torch.acos(cosine_similarity)
        spectral_angle = torch.nan_to_num(cosine_similarity, 1)
        # # When a function perfectly matches, the value is 0
        # # Torch acos is define over [-1,1]
        # # To make this an appropriate loss value, we need to invert the spread and then add 1
        spectral_angle = (-1 * spectral_angle) + 1
        # Average the spectral angle to get the final loss
        loss = torch.mean(spectral_angle)
        return loss

class HybridLoss(nn.Module):
    def __init__(self):
        super(HybridLoss, self).__init__()
        self.MSELoss = nn.MSELoss()
        # Controls the weighting of the importance between MSE and SAM
        self.alpha = 0.1

    def forward(self, y, y_reconstructed):
        return self.forward_mse(y,y_reconstructed) + self.alpha * self.forward_sam(y, y_reconstructed)

    def forward_sam(self, y, y_reconstructed):
        # Normalize y and y_reconstructed along the feature dimension
        # This should normalize along the length of the spectral vector
        epsilon = 1e-8
        normalize_r = torch.sqrt(torch.sum(y_reconstructed**2, dim=1)) + epsilon
        # This should also normalize along the length of the spectral vector
        normalize_t = torch.sqrt(torch.sum(y**2, dim=1))
        # Compute cosine similarity between the normalized vectors
        # This is computer
        numerator = torch.sum((y * y_reconstructed), dim=1)
        denominator = normalize_r * normalize_t
        cosine_similarity = numerator / denominator
        # Compute the spectral angle in radians
        spectral_angle = torch.acos(cosine_similarity)
        spectral_angle = torch.nan_to_num(cosine_similarity, 1)
        # When a function perfectly matches, the value is 0
        # Torch acos is define over [-1,1]
        # To make this an appropriate loss value, we need to invert the spread and then add 1
        spectral_angle = (-1 * spectral_angle) + 1
        # Average the spectral angle to get the final loss
        loss = torch.mean(spectral_angle)
        return loss
    
    def forward_mse(self, y, y_reconstructed):
        return self.MSELoss.forward(y, y_reconstructed)

### Utility Functions

In [5]:
# Initialize the model and the Skorch wrapper
def create_skorch_model(encoding_dim: int, wave: int, large=True, loss='MSE', use_tensorboard=True) -> NeuralNetRegressor:
    if large:
        autoencoder = Autoencoder(encoding_dim, wave)
    else:
        autoencoder = AutoencoderSmall(encoding_dim, wave)
    # Choose loss function
    loss_lookup = {
        'MSE': nn.MSELoss,
        'SAM': CosSpectralAngleLoss,
        'Hybrid': HybridLoss
    }
    try:
        loss_fnc = loss_lookup[loss]
    except KeyError as e:
        print('Invalid loss function!')
        sys.exit(1)
    print(f'Using {loss} as loss!')
    # Add a monitor to save the best performance
    monitor = lambda net: all(net.history[-1, ('train_loss_best', 'valid_loss_best')])
    checkpoint = Checkpoint(monitor=monitor, f_params="params_{last_epoch[epoch]}.pt")
    # Create initial callbacks
    callbacks = [ProgressBar(),early_stopping, checkpoint]
    # Enable us to view model performance through the TensorBoard GUI
    if use_tensorboard:
        from torch.utils.tensorboard import SummaryWriter
        from skorch.callbacks import TensorBoard
        writer = SummaryWriter()
        callbacks.append(TensorBoard(writer))
    skorch_model = NeuralNetRegressor(
        autoencoder,
        max_epochs=100,    # adjust as needed
        lr=0.01,
        optimizer = torch.optim.Adam,
        criterion = loss_fnc, # Chose loss function class here
        callbacks = callbacks,
        batch_size = 7, # This will need to vary based on the computer utilized
        iterator_train__shuffle=True,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    return skorch_model

def batch_predict(model, dataloader):
    pass


# Predict some sample images here
def visualize_model_performance(autoencoder: NeuralNetRegressor, cube_data: dict, threshold: float=0.15, title='', factor=1):
    # Make a single figure
    samples = len(cube_data.keys())
    i = 0
    fig, ax = plt.subplots(samples, 2, figsize=(15, 5*samples))
    for name, cube in tqdm(cube_data.items()):
        # Grab the correct pixels
        input_pixels = cube['data'][::10,::10,:].reshape(-1, cube['data'][::10,::10,:].shape[-1]).astype(np.float32) / factor
        start_time = time.time_ns()
        predicted_pixels = autoencoder.predict(input_pixels)
        stop_time = time.time_ns()
        print(f'Predicted reconstruction error in {(stop_time-start_time)/10**9} seconds')
        predicted_cube = predicted_pixels.reshape(cube['data'][::10,::10,:].shape)
        # Calculate L2 error and visualize
        # Plot RGB, ground truth, and predicted labels side by side
        error_img = np.linalg.norm(predicted_cube-(cube['data'][::10,::10,:] / factor), axis=2)
        im = ax[i, 0].imshow(error_img)
        ax[i, 0].set_title(f'Reconstruction Loss for Scene: {name}')
        fig.colorbar(im, label='Reconstruction Error')
        image = cv2.cvtColor((cube['data']/factor)[:,:,:3].astype(np.float32), cv2.COLOR_BGR2RGB)
        ax[i, 1].imshow(image)
        ax[i, 1].set_title('RGB Representation')
        # Increment the row
        i+= 1
    plt.suptitle(f'Autoencoder Performance for {title}', y=1.01)
    plt.tight_layout()
    plt.show()

def save_model(model: NeuralNetRegressor, file_name='.'):
    '''
    Save model to file
    '''
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

def load_model(model: NeuralNetRegressor, file_name='.') -> NeuralNetRegressor:
    '''
    Load autoencoder and return object
    '''
    with open(file_name, 'rb') as f:
        model  = pickle.load(f)
        return model

In [6]:
def create_inference_png(img, pred, image_name, max_pred, plot_thresholds=[0.001, 0.002,0.005, 0.01,0.02,0.025, 0.05]):
    nrows = 2 + len(plot_thresholds)
    fig_height = 6 + len(plot_thresholds)
    fig, ax = plt.subplots(nrows, 2, tight_layout=True, dpi=300, figsize=(10, fig_height))
    a_map = pred
    a_map = a_map + abs(a_map.min())
    if a_map.max() > 0:
        a_map = a_map / (max_pred + abs(a_map.min()))

    if img.shape[2] > 3:
        ir = img[:,:,3:]
        rgb = img[:,:,[2,1,0]]

    ax[0][0].imshow((rgb * 255).astype(np.uint8), vmax=255, vmin=0)
    ax[0][1].imshow((ir* 255).astype(np.uint8), vmax=255, vmin=0)
    ax[0][0].set_title('RGB')
    ax[0][1].set_title('IR')
    ax[1][0].imshow((a_map * 255).astype(np.uint8), vmax=255, vmin=0)
    ax[1][0].set_title('anomaly_map ')
    overlay = ir.copy()
    mask = a_map > np.array(plot_thresholds).min()
    overlay[mask,1] = a_map[mask]

    ax[1][1].imshow((overlay * 255).astype(np.uint8), vmax=255, vmin=0)
    ax[1][1].set_title('IR_overlay')
    for i, threshold in enumerate(plot_thresholds):
        a_map_threshold = a_map.copy()
        a_map_threshold[a_map_threshold < threshold] = 0
        a_map_threshold[a_map_threshold >= threshold] = 1
        ax[2 + i][0].set_title(f'anomaly_map_threshold: {threshold}')
        overlay = ir.copy()
        overlay[a_map_threshold == 1] = [1, 0, 0]
        ax[2 + i][0].imshow((a_map_threshold * 255).astype(np.uint8), vmax=255, vmin=0)
        ax[2 + i][1].imshow((overlay * 255).astype(np.uint8), vmax=255, vmin=0)


        ax[2 + i][1].set_title('IR_overlay with threshold')
    fig.suptitle(image_name)
    return fig, ax


### Create Dataloader

In [7]:
hsi_data = CubeDataset('/home/nathaniel/Downloads/yogurt_data_deploy/train_eval/*/*', subset=10, eval=False, RGB=False, SWIR=True, crop=300)

Training from 1363 samples
Found 10 dark reference files
Calculated dark reference with a size of (180, 430, 6)
Found 39 white reference files
Calculated white reference with a size of (180, 430, 6)


### Train Model with Full Dataset

In [8]:
autoencoder_large_hybrid = create_skorch_model(1, 3, large=True, loss='Hybrid')
autoencoder_large_hybrid.fit(hsi_data, None)

Using Hybrid as loss!


2025-02-18 09:14:44.219146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739888084.288242    6620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739888084.308028    6620 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 09:14:44.446804: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


  0%|          | 0/195 [00:00<?, ?it/s]

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1        0.0021        0.0004     +  1152.2229


  0%|          | 0/195 [00:00<?, ?it/s]

...

  0%|          | 0/195 [00:00<?, ?it/s]

     11        0.0003        0.0004        280.2650


  0%|          | 0/195 [00:00<?, ?it/s]

Stopping since valid_loss has not improved in the last 10 epochs.


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Autoencoder(
    (encoder): Sequential(
      (0): Linear(in_features=3, out_features=150, bias=True)
      (1): ReLU()
      (2): Linear(in_features=150, out_features=100, bias=True)
      (3): ReLU()
      (4): Linear(in_features=100, out_features=75, bias=True)
      (5): ReLU()
      (6): Linear(in_features=75, out_features=50, bias=True)
      (7): ReLU()
      (8): Linear(in_features=50, out_features=1, bias=True)
    )
    (decoder): Sequential(
      (0): Linear(in_features=1, out_features=50, bias=True)
      (1): ReLU()
      (2): Linear(in_features=50, out_features=75, bias=True)
      (3): ReLU()
      (4): Linear(in_features=75, out_features=100, bias=True)
      (5): ReLU()
      (6): Linear(in_features=100, out_features=3, bias=True)
      (7): Sigmoid()
    )
  ),
)

In [9]:
import dill as pickle
save_model(autoencoder_large_hybrid, './yogurt_models/autoencoder_large_hybrid_production_02_13_25_swir.pkl')
autoencoder_large_hybrid.save_params(f_params='autoencoder_large_hybrid_production_02_13_25_swir_params.pkl')

In [12]:
# import dill as pickle
autoencoder_large_hybrid = create_skorch_model(3, 6, large=True, loss='Hybrid')
autoencoder_large_hybrid.initialize()
autoencoder_large_hybrid.load_params(f_params='autoencoder_large_hybrid_production_02_09_25_params.pkl')

Using Hybrid as loss!


In [10]:
# Create testing dataset
# hsi_data = CubeDataset('/home/nathaniel/Downloads/yogurt_data_deploy/playtime/*/*', subset=10, eval=True)
hsi_data = CubeDataset('/home/nathaniel/Downloads/yogurt_data_deploy/playtime/*/*', subset=10, eval=True)

Evaluating 138 samples
Found 35 dark reference files
Calculated dark reference with a size of (240, 490, 6)
Found 70 white reference files
Calculated white reference with a size of (240, 490, 6)


### Feed Dataset through Model

In [11]:
# test = []
# rgb = []
for sample in tqdm(hsi_data):
    pred = autoencoder_large_hybrid.predict(sample[0])
    pred = pred.reshape(sample[2])
    error_img = np.linalg.norm(pred-sample[0].reshape(sample[2]), axis=2)
    # test.append(error_img)
    # Save the data
    save_name = sample[1][:-4] + '_prediction_02_18_swir.npy'
    # rgb.append(cv2.imread(sample[1][:-4] + '.jpg'))
    np.save(save_name, error_img)

  0%|          | 0/138 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x6 and 3x150)

### Run the RGB Ablation Results

In [6]:
autoencoder_large_hybrid = create_skorch_model(3, 3, large=True, loss='Hybrid')
autoencoder_large_hybrid.initialize()
autoencoder_large_hybrid.load_params(f_params='autoencoder_large_hybrid_production_02_11_25__rgb_params.pkl')

Using Hybrid as loss!


2025-02-12 18:16:58.276951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739402218.320081    9398 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739402218.332726    9398 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 18:16:58.438403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
hsi_data = CubeDataset('/home/nathaniel/Downloads/yogurt_data_deploy/train_eval/*/*', subset=10, eval=True)

Evaluating 1571 samples
Found 10 dark reference files
Calculated dark reference with a size of (240, 490, 6)
Found 39 white reference files
Calculated white reference with a size of (240, 490, 6)


In [14]:
# test = []
# rgb = []
for sample in tqdm(hsi_data):
    pred = autoencoder_large_hybrid.predict(sample[0][:,3:])
    pred = pred.reshape(*sample[2][0:2],3)
    error_img = np.linalg.norm(pred-sample[0][:,3:].reshape((*sample[2][0:2],3)), axis=2)
    # test.append(error_img)
    # Save the data
    save_name = sample[1][:-4] + '_prediction_02_18_swir.npy'
    # rgb.append(cv2.imread(sample[1][:-4] + '.jpg'))
    np.save(save_name, error_img)

  0%|          | 0/1571 [00:00<?, ?it/s]

In [15]:
dark_ref = np.mean(np.array([np.load(z)[::10,::10,:] for z in glob.glob('/home/nathaniel/Downloads/yogurt_data_deploy/*/*/*ref_dark*.npy')]), axis=0)
white_ref = np.mean(np.array([np.load(z)[::10,::10,:] for z in glob.glob('/home/nathaniel/Downloads/yogurt_data_deploy/*/*/*ref_white*.npy')]), axis=0)

In [16]:
def reflectance(cube, dark_ref, white_ref):
    return np.clip(
        np.nan_to_num((cube - dark_ref) / (white_ref/0.40 - dark_ref)),
        0, 1.00).astype(np.float32)

In [17]:
all_preds = glob.glob('/home/nathaniel/Downloads/yogurt_data_deploy/*/*/*prediction_02_18_swir.npy')
thresh = [0.01,0.02,0.025, 0.05, 0.1, 0.2, 0.5]
for pred in tqdm(all_preds):
    # Load image file
    pred_img = np.load(pred)
    # Load the original cube
    cube_path = f"{pred.split('_prediction')[0]}.npy"
    print(cube_path)
    cube = np.load(cube_path)[::10,::10,:]
    cube = reflectance(cube, dark_ref, white_ref)
    # Generate the inference plot
    image_name = pred.split('/')[-1][:-4]
    save_path = f"/home/nathaniel/cubert/internal_rnd_applications/yogurt/results/{image_name}.png"
    fig, ax = create_inference_png(cube, pred_img, image_name, 1.0, plot_thresholds=thresh)
    fig.savefig(save_path)
    plt.close(fig)
    plt.ioff()
    gc.collect()

  0%|          | 0/1709 [00:00<?, ?it/s]

/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_121032_frame_31_ok_ok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_122146_frame_42_ok_nok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_121631_frame_37_ok_nok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_115901_frame_22_ok_ok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_123003_frame_53_nok_ok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_122715_frame_48_ok_ok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_120727_frame_29_ok_ok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_123223_frame_55_nok_ok_rd4_rw9.npy
/home/nathaniel/Downloads/yogurt_data_deploy/playtime/friday_playtime/20250121_122413_frame_